In [16]:
from source import data_import
from source.constants import CHANNELS
import pandas as pd


In [17]:
# p_df = data_import.import_patients(patient_ids=[1, 2, 3, 4], seizure_flag=True)
# p_df.head()

In [18]:
# data_import.save_pyarrow(p_df, file_name="preprocess_test")

In [19]:
p_df = data_import.load_pyarrow(file_name="preprocess_test")

/home/weasel/reps/ai-seizure-detectives/source/../data/preprocess_test.arrow was loaded.


In [20]:
from source.filter_eeg_channels import filter_eeg_channels

exclude_ranges=[[58, 62], [118, 122]]
fit_df = filter_eeg_channels(p_df, CHANNELS, fs=256, exclude_ranges=exclude_ranges, Q=30)
fit_df.head()

channel,F4-C4,F3-C3,FT9-FT10,FZ-CZ,F7-T7,FP2-F4,T8-P8-1,T8-P8-0,FP1-F3,CZ-PZ,before_seizure,is_seizure
0 days 00:00:00,-30.700675,2.790970,-6.246458,-12.625819,26.713574,-11.828398,-40.801330,-40.801330,-0.398710,58.078766,False,False
0 days 00:00:00.003906250,-10.319968,1.083164,-1.993871,-4.165896,9.228259,-3.894393,-13.759008,-13.759008,-0.002848,19.907382,False,False
0 days 00:00:00.007812500,-2.200027,0.394352,-0.305718,-0.799886,2.247480,-0.738115,-2.982458,-2.982458,0.147268,4.677136,False,False
0 days 00:00:00.011718750,-13.337263,1.442223,-2.883424,-5.217581,12.188861,-5.125166,-18.140539,-18.140539,0.009338,26.544704,False,False
0 days 00:00:00.015625,15.164319,-1.099635,3.174129,6.435789,-12.652102,5.960205,19.951526,19.951526,0.440694,-27.708167,False,False


In [21]:
### go through non-seizure data (before_seizure)
# extract seizure-starts
    # for each seizure start
        # get previous 20 seconds
        # check if is_seizure in it AND if all are before_seizure
            # yes: drop and continue to next seizure start
        # take 20 seconds:
            # mean of amplitude over time per channel
            # mean of power spectrum for frequency ranges per channel

### go through seizure data (is_seizure)
# take seizre-starts
    # for each seizure start
        # get next 20 seconds
        # check if all are is_seizure
            # no: drop and continue
        # take 20 seconds:
            # mean of amplitude over time per channel
            # mean of power spectrum for frequency ranges per channel

# out put: data resampled to 20sec with features

In [22]:
fit_df.info()

<class 'pandas.core.frame.DataFrame'>
TimedeltaIndex: 24484608 entries, 0 days 00:00:00 to 1 days 02:34:02.996093750
Freq: 3906250N
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   F4-C4           float64
 1   F3-C3           float64
 2   FT9-FT10        float64
 3   FZ-CZ           float64
 4   F7-T7           float64
 5   FP2-F4          float64
 6   T8-P8-1         float64
 7   T8-P8-0         float64
 8   FP1-F3          float64
 9   CZ-PZ           float64
 10  before_seizure  bool   
 11  is_seizure      bool   
dtypes: bool(2), float64(10)
memory usage: 2.1 GB


In [23]:
fit_df['seizure_start'] = fit_df['is_seizure'] & ~fit_df['is_seizure'].shift(fill_value=False)
fit_df.loc[fit_df['seizure_start'] == True, 'seizure_start']

0 days 00:49:55    True
0 days 01:24:26    True
0 days 02:28:51    True
0 days 03:16:54    True
0 days 04:28:39    True
0 days 05:05:26    True
0 days 06:31:01    True
0 days 07:28:16    True
0 days 07:40:54    True
0 days 08:50:52    True
0 days 09:00:45    True
0 days 10:06:54    True
0 days 11:01:55    True
0 days 12:30:45    True
0 days 13:27:45    True
0 days 14:37:55    True
0 days 15:23:28    True
0 days 18:04:47    True
0 days 20:21:05    True
0 days 23:01:38    True
0 days 23:36:41    True
Name: seizure_start, dtype: bool

In [24]:
fit_df['seizure_id'] = fit_df['seizure_start'].cumsum()
fit_df['seizure_id'].tail()

1 days 02:34:02.980468750    21
1 days 02:34:02.984375       21
1 days 02:34:02.988281250    21
1 days 02:34:02.992187500    21
1 days 02:34:02.996093750    21
Freq: 3906250N, Name: seizure_id, dtype: int64

In [25]:
fit_df.index

TimedeltaIndex([          '0 days 00:00:00', '0 days 00:00:00.003906250',
                '0 days 00:00:00.007812500', '0 days 00:00:00.011718750',
                   '0 days 00:00:00.015625', '0 days 00:00:00.019531250',
                '0 days 00:00:00.023437500', '0 days 00:00:00.027343750',
                   '0 days 00:00:00.031250', '0 days 00:00:00.035156250',
                ...
                '1 days 02:34:02.960937500', '1 days 02:34:02.964843750',
                   '1 days 02:34:02.968750', '1 days 02:34:02.972656250',
                '1 days 02:34:02.976562500', '1 days 02:34:02.980468750',
                   '1 days 02:34:02.984375', '1 days 02:34:02.988281250',
                '1 days 02:34:02.992187500', '1 days 02:34:02.996093750'],
               dtype='timedelta64[ns]', length=24484608, freq='3906250N')

In [26]:
# extract seizure-stab4seizure_intervals
    # for each seizure start

fit_df['segment_id'] = 0

b4_seizure = []
# for time, seizure in (s_df[s_df == True]).items():
for time, seizure in fit_df.loc[fit_df['seizure_start'] == True, 'seizure_start'].items():
    intervals = []
    segment_id = 15
    while True:
        # get previous 20 seconds
        int_end = time - fit_df.index.freq
        int_start = time - (pd.Timedelta(seconds=20))
        
        if int_start < pd.Timedelta('0S'):
            raise('this should probably not happen.')
        b4s_int = fit_df.loc[int_start:int_end, :].copy()
        b4s_int['segment_id'] = segment_id
    
        # check if is_seizure in it AND if all are before_seizure
        # s_int.loc['10S', 'is_seizure'] = True  # check
        if any(b4s_int['is_seizure']) or not all(b4s_int['before_seizure']):
            print(f"either previous seizure found or start of before seizure interval reached. seizure number: {b4s_int['seizure_id'].values[0]}")
            break # get out of while loop
            
        intervals.insert(0, b4s_int)
        time = int_start
        segment_id -= 1
    
    b4_seizure.extend(intervals)

b4seizure_df = pd.concat(b4_seizure)

# b4seizure_df.info()


either previous seizure found or start of before seizure interval reached. seizure number: 0
either previous seizure found or start of before seizure interval reached. seizure number: 1
either previous seizure found or start of before seizure interval reached. seizure number: 2
either previous seizure found or start of before seizure interval reached. seizure number: 3
either previous seizure found or start of before seizure interval reached. seizure number: 4
either previous seizure found or start of before seizure interval reached. seizure number: 5
either previous seizure found or start of before seizure interval reached. seizure number: 6
either previous seizure found or start of before seizure interval reached. seizure number: 7
either previous seizure found or start of before seizure interval reached. seizure number: 8
either previous seizure found or start of before seizure interval reached. seizure number: 9
either previous seizure found or start of before seizure interval reac

In [27]:
b4seizure_df.shape

(1566720, 15)

### Seizures

In [28]:
fit_df['segment_id'] = 0

post_seizure = []
# for time, seizure in (s_df[s_df == True]).items():
for time, seizure in fit_df.loc[fit_df['seizure_start'] == True, 'seizure_start'].items():
    seizure_length = fit_df['seizure_start']
    intervals = []
    segment_id = 1
    while True:
        # get previous 20 seconds
        int_start = time  
        int_end = int_start + (pd.Timedelta(seconds=20))
        if int_start > fit_df.index[-1]:
            print('reached end of data.')
            break
        
        post_int = fit_df.loc[int_start: int_end, :].copy()
        post_int['segment_id'] = segment_id
    
        # check if is_seizure in it AND if all are before_seizure
        # s_int.loc['10S', 'is_seizure'] = True  # check
        if not all(post_int['is_seizure']):
            print(f"seizure end reached. seizure number: {post_int['seizure_id']}, number of segments completed: {segment_id-1}")
            break # get out of while loop

        intervals.append(post_int)
        time =  int_end + fit_df.index.freq
        segment_id += 1
    post_seizure.extend(intervals)

post_seizure_df = pd.concat(post_seizure)

seizure end reached. seizure number: 0 days 00:50:35.007812500    1
0 days 00:50:35.011718750    1
0 days 00:50:35.015625       1
0 days 00:50:35.019531250    1
0 days 00:50:35.023437500    1
                            ..
0 days 00:50:54.992187500    1
0 days 00:50:54.996093750    1
0 days 00:50:55              1
0 days 00:50:55.003906250    1
0 days 00:50:55.007812500    1
Freq: 3906250N, Name: seizure_id, Length: 5121, dtype: int64, number of segments completed: 2
seizure end reached. seizure number: 0 days 01:24:46.003906250    2
0 days 01:24:46.007812500    2
0 days 01:24:46.011718750    2
0 days 01:24:46.015625       2
0 days 01:24:46.019531250    2
                            ..
0 days 01:25:05.988281250    2
0 days 01:25:05.992187500    2
0 days 01:25:05.996093750    2
0 days 01:25:06              2
0 days 01:25:06.003906250    2
Freq: 3906250N, Name: seizure_id, Length: 5121, dtype: int64, number of segments completed: 1
seizure end reached. seizure number: 0 days 02:29:31.007

In [29]:
merged_df = pd.concat(objs=[b4seizure_df, post_seizure_df])
merged_df.shape

(1884222, 15)

In [30]:
data_import.save_pyarrow(merged_df, file_name="preprocessed_df")

/home/weasel/reps/ai-seizure-detectives/source/../data/preprocessed_df.arrow was successfully written.
